In [7]:
import pandas as pd
import sqlite3
from Levenshtein import distance

# connect to the SQLite database and read the data into a pandas dataframe
conn = sqlite3.connect('data_files/words.db')
df = pd.read_sql('SELECT * FROM words', conn)

# close the database connection
conn.close()

In [2]:
df['Length_word'] = df['word'].str.len()
df['Length_translation'] = df['translation'].str.len()
# Calculate Levenshtein Distance between word and translation
df['Distance'] = df.apply(lambda row: distance(row['word'], row['translation']), axis=1)

In [3]:
# Calculate word lengths and divide into three bins
#df['Difficulty'] = pd.cut(df['Length'], bins=3, labels=['Easy', 'Medium', 'Hard'])

In [4]:
# dividing into three quantiles
df['d1'] = pd.qcut(df['Length_word'], q=3, labels=[0, 1, 2])
df['d1'] = df['d1'].astype(int)
df['d2'] = pd.qcut(df['Length_translation'], q=3, labels=[0, 1, 2])
df['d2'] = df['d2'].astype(int)
df['d3'] = pd.qcut(df['Distance'], q=3, labels=[0, 1, 2])
df['d3'] = df['d3'].astype(int)

In [5]:
df['difficulty'] = df['d1'] + df['d2'] + df['d3']
df.loc[df['Distance'] == 0, 'difficulty'] = 0

df = df.loc[:, ['word', 'type', 'translation', 'example', 'example_translation',
       'russian', 'appear', 'trial_d', 'trial_r', 'success', 'weight', 'difficulty']]

In [6]:
writer = pd.ExcelWriter('data_files/xxxxx.xlsx')
df.to_excel(writer, sheet_name='update')
writer.close()

In [9]:
from fuzzywuzzy import fuzz

# Calculate similarity between word and translation using fuzzywuzzy
df['Similarity'] = df.apply(lambda row: fuzz.ratio(row['word'], row['translation']), axis=1)
df['Distance'] = df.apply(lambda row: distance(row['word'], row['translation']), axis=1)

writer = pd.ExcelWriter('data_files/xx2.xlsx')
df.to_excel(writer, sheet_name='update')
writer.close()